In [2]:
pip install undetected-chromedriver

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47129 sha256=7955d79bea088679a59e371ca23e6c2c778bb6563554b649603852f07c6e3e85
  Stored in directory: /Users/felipesilverio/Library/Caches/pip/wheels/d1/9c/b4/ccdca75ca356899a9b65190f048a03058f1fe23602c642b88e
Successfully built undetected-chromedriver

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import mysql.connector
import env
import numpy as np
import re

HOST= env.HOST
USERNAME= env.DBUSER
DBNAME= env.DBNAME
PASSWORD= env.PASSWORD
USER= env.LGGUSER

In [ ]:
def ETL_Bluebook():

    def __init__(self):

        self.cols = ['summary_date','summary_netsales_total', 'summary_tktcount_total', 'summary_tktcount_ontime', 'tktcount_ontime_pct',
                     'tkt_avg','av_time_all','av_time_callin','av_time_counter','av_time_stall','av_time_dt','av_time_patio',
                     'av_time_oa','av_time_delivery','reply_time','cash_due','cash_deposits', 'summary_overshort_amount',
                     'cancel_tkt','cancel_tkt_cnt','crew_labor','total_labor', 'food_cost','food_cost_pct']

        self.cols_to_update=['stores_number','summary_date','summary_netsales_total', 
                             'summary_tktcount_total', 'summary_tktcount_ontime','summary_overshort_amount']
        
        self.df = []
        
    def is_date(value):
        bool(re.match(r'^\d{2}-\d{2}-\d{2}$', str(value)))

    def extract(self):
        df = pd.read_excel('/Users/felipesilverio/Documents/GitHub/miscellanious/BlueBook_OSonic_TEMP.xlsx', header=4)
        df.columns = self.cols
        df.drop(0, inplace=True)
        self.df = df.copy()

    def transform(self):
        df = self.df.copy()
        # Apply function to create new column
        df['is_date'] = df['summary_date'].apply(is_date).astype(int)
        df['summary_date'] = df['summary_date'].apply(lambda x: x if is_date(x) else str(x)[:4])

        df['stores_number'] = df.summary_date
        for idx in df.index:
            val = df.loc[idx, 'summary_date']
            if val.isdigit():  # If it's a numeric string (like 1082 or 1130)
                # Fill this value for the next 7 rows (including itself if needed)
                df.loc[idx:idx+7, 'stores_number'] = val
        df = df[df.is_date==1].copy()


        df['summary_date'] = pd.to_datetime(df['summary_date'], format='%m-%d-%y', errors='coerce')

        df.reset_index(drop=True)
        self.df = df[self.cols_to_update]

    def load(self):
        df = self.df.copy()

        conn = mysql.connector.connect(user=USERNAME, password=PASSWORD, host=HOST, database=DBNAME, allow_local_infile=True)
        cursor = conn.cursor()

        # df = pd.read_csv('/Users/felipesilverio/Documents/GitHub/miscellanious/Consolidate_BB_v2.csv')
        df = df.dropna(axis=1, how='all')

        # Step 1: Create temporary table

        cursor.execute("""
            CREATE TABLE CMGSOAR.update_data (
                stores_number VARCHAR(28),
                summary_date VARCHAR(28),
                summary_netsales_total VARCHAR(28),
                summary_tktcount_total VARCHAR(28),
                summary_tktcount_ontime VARCHAR(28),
                summary_overshort_amount VARCHAR(28)
        );
        """)

        self.df.to_csv('consolidate_bb.csv')

        # Step 2: Load data into temporary table
        values = df.values.tolist()

        values = [[None if isinstance(val, float) and np.isnan(val) else val for val in row] for row in values]
        columns2 = ', '.join(df.columns)
        placeholders = ', '.join(['%s'] * len(df.columns))

        sql = f"""INSERT INTO CMGSOAR.update_data ({columns2}) VALUES ({placeholders})"""
        cursor.executemany(sql, values)

        cursor.execute("""
            UPDATE CMGSOAR.store_summary AS target
            JOIN CMGSOAR.update_data AS upd
            ON target.stores_number = upd.stores_number
            AND target.summary_date = upd.summary_date
            SET 
                target.summary_netsales_total = upd.summary_netsales_total,
                target.summary_tktcount_total = upd.summary_tktcount_total,
                target.summary_tktcount_ontime = upd.summary_tktcount_ontime,
                target.summary_overshort_amount = upd.summary_overshort_amount;
        """)

        cursor.execute("""
            DROP TABLE CMGSOAR.update_data;
        """)

        # Commit and close
        conn.commit()
        cursor.close()
        conn.close()

    def run(self):
        self.extract()
        self.transform()
        self.load()


In [7]:
cols = ['summary_date','summary_netsales_total', 'summary_tktcount_total', 'summary_tktcount_ontime', 'tktcount_ontime_pct',
        'tkt_avg','av_time_all','av_time_callin','av_time_counter','av_time_stall','av_time_dt','av_time_patio',
        'av_time_oa','av_time_delivery','reply_time','cash_due','cash_deposits', 'summary_overshort_amount',
        'cancel_tkt','cancel_tkt_cnt','crew_labor','total_labor', 'food_cost','food_cost_pct']

cols_to_update=['stores_number','summary_date','summary_netsales_total', 'summary_tktcount_total', 'summary_tktcount_ontime',
                'summary_overshort_amount']

# Function to check if string is in date format (MM-DD-YY)
def is_date(value):
    if isinstance(value, str):
        return bool(re.match(r'^\d{2}-\d{2}-\d{2}$', value))
    return False  # safely handle None or other types
     

In [8]:
df = pd.read_excel('/Users/felipesilverio/Documents/GitHub/Bluebook_etl_lambda/tmp/downloaded_files/BlueBook_OSonic_TEMP.xlsx', header=4)
df.columns = cols
df.drop(0, inplace=True)

# Apply function to create new column
df['is_date'] = df['summary_date'].apply(is_date).astype(int)
df['summary_date'] = df['summary_date'].apply(lambda x: x if is_date(x) else str(x)[:4])

df['stores_number'] = df.summary_date
for idx in df.index:
    val = df.loc[idx, 'summary_date']
    if val.isdigit():  # If it's a numeric string (like 1082 or 1130)
        # Fill this value for the next 7 rows (including itself if needed)
        df.loc[idx:idx+7, 'stores_number'] = val

df = df[df.is_date==1].copy()

df['summary_date'] = pd.to_datetime(df['summary_date'], format='%m-%d-%y', errors='coerce')

df.reset_index(drop=True)
df = df[cols_to_update].copy()
df.to_csv('consolidate_bb.csv')



/Users/felipesilverio/Documents/GitHub/Bluebook_etl_lambda/.venv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
conn = mysql.connector.connect(user=USERNAME, password=PASSWORD, host=HOST, database=DBNAME, allow_local_infile=True)
cursor = conn.cursor()

# df = pd.read_csv('/Users/felipesilverio/Documents/GitHub/miscellanious/Consolidate_BB_v2.csv')
df = df.dropna(axis=1, how='all')

# Step 1: Create temporary table

cursor.execute("""
    CREATE TABLE CMGSOAR.update_data (
        stores_number VARCHAR(28),
        summary_date VARCHAR(28),
        summary_netsales_total VARCHAR(28),
        summary_tktcount_total VARCHAR(28),
        summary_tktcount_ontime VARCHAR(28),
        summary_overshort_amount VARCHAR(28)
);
""")

# Step 2: Load data into temporary table
values = df.values.tolist()

values = [[None if isinstance(val, float) and np.isnan(val) else val for val in row] for row in values]
columns2 = ', '.join(df.columns)
placeholders = ', '.join(['%s'] * len(df.columns))

sql = f"""INSERT INTO CMGSOAR.update_data ({columns2}) VALUES ({placeholders})"""
cursor.executemany(sql, values)

cursor.execute("""
    UPDATE CMGSOAR.store_summary AS target
    JOIN CMGSOAR.update_data AS upd
    ON target.stores_number = upd.stores_number
    AND target.summary_date = upd.summary_date
    SET 
        target.summary_netsales_total = upd.summary_netsales_total,
        target.summary_tktcount_total = upd.summary_tktcount_total,
        target.summary_tktcount_ontime = upd.summary_tktcount_ontime,
        target.summary_overshort_amount = upd.summary_overshort_amount;
""")

cursor.execute("""
    DROP TABLE CMGSOAR.update_data;
""")

# Commit and close
conn.commit()
cursor.close()
conn.close()
